<a href="https://colab.research.google.com/github/fblazejewski/Przewidywanie-sredniej-filmow/blob/master/Projekt_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
#Klasa przechowująca dane + parser stron tworzący baze filmów

In [0]:
import requests
import re
import datetime
import pandas as pd
import json
import sys

class Data:
  frame = pd.DataFrame()

  def createData(self, limit=0):
    links = self._grabLinks(limit)
    for link in links:
      newDF = self._getDataFrame(link)
      if newDF is not None:
        self.frame = pd.concat([self.frame, newDF], ignore_index=True)

  def divideData(self, percent, side):
    if side == 'start':
      return self.frame.loc[:len(self.frame)*percent], self.frame.loc[:len(self.frame)*percent]['Średnia ocen widzow']
    elif side == 'end':
      return self.frame.loc[len(self.frame)*percent:], self.frame.loc[len(self.frame)*percent:]['Średnia ocen widzow']


  def _grabLinks(self, limit):
    sitemap = requests.get('https://www.rottentomatoes.com/sitemap.xml').text
    usableLinks = []

    for maplink in re.findall('<loc>(.*?)</loc>', sitemap):

      moviesList = requests.get(maplink).text

      for link in re.findall('<loc>(.*?)</loc>', moviesList):
        #zapisz tylko 'przydatne' linki
        if re.search('(/pictures/|/trailers/)', link) is None:
          usableLinks.append(link)
          if (limit > 0 and len(usableLinks) >= limit):
            return usableLinks
    return usableLinks


  def _getDataFrame(self, link):
    try:
      #pobieranie tesktu ze strony
      strona = requests.get(link).text

      j = re.findall('<script type="application/ld\+json">(.*?)</script>', strona, re.M | re.S)
      if j is None:
        raise IndexError("brak danych json!")
      if len(j) == 0:
        raise IndexError("pusty json!")

      data = json.loads(j[0].strip())


      #tytul:
      tytul = data['name']

      #gatunek:
      gatunek = data['genre']

      #autor:
      autorzy = []
      for autor in data['author']:
        autorzy.append(autor['name'])

      #reżyserowie:
      rezyserowie = []
      for rezyser in data['director']:
        rezyserowie.append(rezyser['name'])

      #wydawca:
      wydawca = data['productionCompany']['name']

      #premiera:
      premiera = re.findall( r'<time datetime=\"(\d.*?)T\d\d', strona, re.S|re.M)
      # zwraca coś takiego: 2019-12-19 (rok-miesiąc-dzień), konwersja w pythonie:
      premiera = datetime.datetime.strptime(premiera[0], "%Y-%m-%d")

      #długość:
      dlugosc = re.findall( r'time datetime=\"P(\d{1,3})M', strona, re.S|re.M)
      if len(dlugosc) == 0:
        raise IndexError("brak dlugosci")
      dlugosc = int(dlugosc[0])

      #obsada:
      obsada = []
      for aktor in data['actors']:
        obsada.append(aktor['name'])
      if len(obsada) == 0:
        raise IndexError("brak obsady")

      #średnia ocen (rotten tomato ma procentowy wynik):
      srednia_ocen = re.findall( r'mop-ratings-wrap__percentage\">\s+(\d{1,3})%', strona, re.S|re.M)
      if len(srednia_ocen) == 0:
        raise IndexError("brak sredniej ocen!")
      srednia_krytykow = int(srednia_ocen[0])
      srednia_widzow = int(srednia_ocen[1])

      #ilość ocen:
      #dla krytyków:
      ilosc_ocen_krytykow = re.findall( r'<small class=\"mop-ratings-wrap__text--small\">\s+(\d*)', strona, re.S|re.M)
      if len(ilosc_ocen_krytykow) == 0:
        raise IndexError("brak ocen krytykow!")
      ilosc_ocen_krytykow = int(ilosc_ocen_krytykow[0])
      #dla widzów:
      ilosc_ocen_widzow = re.findall( r'(?:Verified|User) Ratings: (.*?)<', strona, re.S|re.M)
      if len(ilosc_ocen_widzow) == 0:
        raise IndexError("brak ocen widzow")
      ilosc_ocen_widzow = ilosc_ocen_widzow[0].replace(',', '')
      ilosc_ocen_widzow = int(ilosc_ocen_widzow)

      lista = [[tytul, gatunek, autorzy, rezyserowie, wydawca, premiera, dlugosc, obsada, srednia_krytykow, ilosc_ocen_krytykow, srednia_widzow, ilosc_ocen_widzow]]
      return pd.DataFrame(lista, columns=['Tytuł', 'Gatunek', 'Autorzy', 'Reżyserowie', 'Wydawca', 'Premiera', 'Długość', 'Obsada', 'Średnia ocen krytyków', 'Ilosc ocen krytykow', 'Średnia ocen widzow', 'Ilosc ocen widzow'])
    except ValueError as ve:
      print(ve, link)
    except IndexError as ie:
      print(ie, link)
    except KeyboardInterrupt:
      sys.exit("awaryjne hamowanie...")
    except:
      print("problem przy linku:", link, "error:", sys.exc_info()[0])

In [0]:
d = Data()
d.createData(3334)
d.frame

In [0]:
d.frame.to_json("filmy.json", orient='split')

---
---

---
#Przygotowanie danych (wczytanych z pliku)

In [0]:
import random
d = Data()
d.frame = pd.read_json("filmy.json", orient='split')
d.frame = d.frame.sample(frac=1).reset_index(drop=True) #mieszanie kolejności

data, results = d.divideData(0.7, 'start')
tests, needed = d.divideData(0.7, 'end')

---
#Tokenizer

In [0]:
from sklearn import preprocessing
import nltk
nltk.download('punkt')

class Tokenizer:
  def __init__(self, data):
    self.data = data
    self.le = preprocessing.LabelEncoder()

  def getGenres(self):
    results = []
    for items in self.data.frame['Gatunek']:
      for arrays in items:
        for word in nltk.word_tokenize(arrays):
          results.append(word)
    return self.le.fit(results).classes_

  def getAuthors(self):
    return self.le.fit(self._flattenList('Autorzy')).classes_

  def getDirectors(self):
    return self.le.fit(self._flattenList('Reżyserowie')).classes_

  def getProducers(self):
    return self.le.fit(self._getStrings('Wydawca')).classes_

  def getActors(self):
    return self.le.fit(self._flattenList('Obsada')).classes_
    
  def _flattenList(self, name):
    flattened = []
    for word in self.data.frame[name]:
      flattened += word
    return flattened    

  def _getStrings(self, name):
    results = []
    for items in self.data.frame[name]:
      results.append(items)
    return results

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


---
#ML

In [0]:
from sklearn import preprocessing
from sklearn import tree
from nltk import word_tokenize
import numpy as np

class ML:
  def __init__(self):
    self.clf = tree.DecisionTreeClassifier()
    self.tokens = Tokenizer(d)
    self.predictions = []
    self.usedData = []


  def teach(self, data, results):
    self.usedData = self._cleanData(data)
    self.clf.fit(self.usedData, results)

  def predict(self, data, results):
    self.predictions = self.clf.predict(self._cleanData(data))
  

  def _cleanData(self, data):
    data = data.reset_index(drop=True)
    data = self._oneHot(data)
    return data.drop(['Tytuł', 'Gatunek', 'Autorzy', 'Reżyserowie', 'Wydawca', 'Obsada', 'Średnia ocen widzow'], axis=1)

  def _oneHot(self, data):
    genres = self.tokens.getGenres()
    producers = self.tokens.getProducers()
    actors = self.tokens.getActors()
    authors = self.tokens.getAuthors()

    for x in range(len(data)):
      #przerób tablice gatunków na słowa
      words = []
      for g in data.at[x, 'Gatunek']:
        for word in word_tokenize(g):
          words.append(word)
      for genre in genres:
        #sprawdź które słowa występują w data
        if genre in words:
          data.at[x, genre] = 1
        else:
          data.at[x, genre] = 0
      
      self._hotInsert(data, x, producers, 'Wydawca')
      self._hotInsert(data, x, authors, 'Autorzy')
      # self._hotInsert(data, x, actors, 'Obsada')
      

    return data

  def _hotInsert(self, data, index, arr, name):
    for word in arr:
      if word in data.at[index, name]:
        data.at[index, word] = 1
      else:
        data.at[index, word] = 0

  def score(self):
    deviant = []
    for i in range(len(self.predictions)):
      deviant.append(abs(needed.iloc[i] - self.predictions[i]))

    print('srednia roznica ocen:', np.mean(deviant))
    a = [0,0,0,0]
    for dev in deviant:
      if dev < 5:
        a[0] += 1
      elif dev < 15:
        a[1] += 1
      elif dev < 20:
        a[2] += 1
      else:
        a[3] += 1
    print("różnica do 5%:", a[0])
    print("różnica do 15%:", a[1])
    print("różnica do 20%:", a[2])
    print("różnica powyżej 20%:", a[3])  

In [0]:
ml = ML()
ml.teach(data, results)

In [0]:
ml.predict(tests, needed)

In [0]:
ml.score()

srednia roznica ocen: 14.393120393120393
różnica do 5%: 171
różnica do 15%: 310
różnica do 20%: 113
różnica powyżej 20%: 220


---
#Testy jednostkowe

In [0]:
import unittest
import pandas as pd

class TestPageParser(unittest.TestCase):
  def testParser(self):
    pars = Data()
    correctResult = pd.DataFrame([["Star Wars: Episode III - Revenge of the Sith",['Action & Adventure', 'Drama', 'Science Fiction & Fantasy'],['George Lucas'],['George Lucas'],"20th Century Fox",datetime.datetime.strptime('2005-05-18', '%Y-%m-%d'),140,['Ewan McGregor', 'Natalie Portman', 'Hayden Christensen', 'Ian McDiarmid', 'Samuel L. Jackson', 'Jimmy Smits', 'Frank Oz', 'Anthony Daniels', 'Christopher Lee', 'Keisha Castle-Hughes', 'James Earl Jones', 'Silas Carson', "Jay Laga'aia", "Jay Lagai'aia", 'Bruce Spence', 'Wayne Pygram', 'Temuera Morrison', 'David Bowers (II) ', 'Mimi Daraphet', 'Paul Davies', 'Oliver Ford Davies', 'Ahmed Best', 'Sandi Finlay', 'Nalini Krishan', 'Rohan Nichol', 'Jeremy Bulloch', 'Amanda Lucas', 'Mary Oyaya', 'Kenny Baker', 'Matt Sloan', 'Orli Shoshan', 'Sandy Thompson', 'Peter Mayhew', 'Marty Wetherill', 'Rebecca Jackson Mendoza', 'Joel Edgerton', 'Bonnie Maree Piesse', 'Jett Lucas', 'Chantal Freer', 'Tux Akindoyeni', 'Matt Rowan', 'Kenji Oates', 'Amy Allen', 'Graeme Blundell', 'Trisha Noble', 'Claudia Karvan', 'Keira Wingate', 'Hayley Mooy', 'Bonnie Piesse', 'Sandy Finlay', 'Bai Ling', 'Katie Lucas', "Genevieve O'Reilly", 'Warren Owens', 'Kee Chan', 'Christopher Kirby', 'Kristy Wright', 'Coinneach Alexander', 'Mousy McCallum', 'Michael Kingma', 'Axel Dench', 'Steven Foy', 'Julian Khazzouh', 'Michael James Rowland', 'Bodie "Tihoi" Taylor', 'David Stiff', 'Robert Cope', 'Matthew Wood', 'Rena Owen'],80,299,66,33683838]], columns=['Tytuł', 'Gatunek', 'Autorzy', 'Reżyserowie', 'Wydawca', 'Premiera', 'Długość', 'Obsada', 'Średnia ocen krytyków', 'Ilosc ocen krytykow', 'Średnia ocen widzow', 'Ilosc ocen widzow'])
    testedResult = pars._getDataFrame('https://www.rottentomatoes.com/m/star_wars_episode_iii_revenge_of_the_sith')
    pd.testing.assert_frame_equal(correctResult, testedResult)
    
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.106s

OK
